In [1]:
# !pip -qq install torchtext==0.3.1
# !git clone https://github.com/MiuLab/SlotGated-SLU.git
!wget -qq https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week08_multitask/conlleval.py



"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda:0')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

np.random.seed(42)
DEVICE

c:\Study\Dialogue_systems\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda', index=0)

In [3]:
import os 

def read_dataset(path):
    with open(os.path.join(path, 'seq.in')) as f_words, \
            open(os.path.join(path, 'seq.out')) as f_tags, \
            open(os.path.join(path, 'label')) as f_intents:
        
        return [
            (words.strip().split(), tags.strip().split(), intent.strip()) 
            for words, tags, intent in zip(f_words, f_tags, f_intents)
        ]
train_data = read_dataset('SlotGated-SLU/data/atis/train/')
val_data = read_dataset('SlotGated-SLU/data/atis/valid/')
test_data = read_dataset('SlotGated-SLU/data/atis/test/')

intent_to_example = {example[2]: example for example in train_data}
for example in intent_to_example.values():
    print('Intent:\t', example[2])
    print('Text:\t', '\t'.join(example[0]))
    print('Tags:\t', '\t'.join(example[1]))
    print()
    
from torchtext.data import Field, LabelField, Example, Dataset, BucketIterator

tokens_field = Field()
tags_field = Field(unk_token=None)
intent_field = LabelField()

fields = [('tokens', tokens_field), ('tags', tags_field), ('intent', intent_field)]

train_dataset = Dataset([Example.fromlist(example, fields) for example in train_data], fields)
val_dataset = Dataset([Example.fromlist(example, fields) for example in val_data], fields)
test_dataset = Dataset([Example.fromlist(example, fields) for example in test_data], fields)

tokens_field.build_vocab(train_dataset)
tags_field.build_vocab(train_dataset)
intent_field.build_vocab(train_dataset)

print('Vocab size =', len(tokens_field.vocab))
print('Tags count =', len(tags_field.vocab))
print('Intents count =', len(intent_field.vocab))

train_iter, val_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, val_dataset, test_dataset), batch_sizes=(32, 128, 128), 
    shuffle=True, device=DEVICE, sort=False
)

Intent:	 atis_flight
Text:	 is	there	a	delta	flight	from	denver	to	san	francisco
Tags:	 O	O	O	B-airline_name	O	O	B-fromloc.city_name	O	B-toloc.city_name	I-toloc.city_name

Intent:	 atis_airfare
Text:	 what	is	the	most	expensive	one	way	fare	from	boston	to	atlanta	on	american	airlines
Tags:	 O	O	O	B-cost_relative	I-cost_relative	B-round_trip	I-round_trip	O	O	B-fromloc.city_name	O	B-toloc.city_name	O	B-airline_name	I-airline_name

Intent:	 atis_airline
Text:	 list	airlines	serving	between	denver	and	san	francisco
Tags:	 O	O	O	O	B-fromloc.city_name	O	B-toloc.city_name	I-toloc.city_name

Intent:	 atis_ground_service
Text:	 tell	me	about	ground	transportation	between	orlando	international	and	orlando
Tags:	 O	O	O	O	O	O	B-fromloc.airport_name	I-fromloc.airport_name	O	B-toloc.city_name

Intent:	 atis_quantity
Text:	 how	many	airlines	have	flights	with	service	class	yn
Tags:	 O	O	O	O	O	O	O	O	B-fare_basis_code

Intent:	 atis_city
Text:	 where	is	lester	pearson	airport
Tags:	 O	O	B-airport_name	

In [4]:
class IntentClassifierModel(nn.Module):
    def __init__(self, vocab_size, intents_count, emb_dim=64, lstm_hidden_dim=128, num_layers=1, dropout_p=0.2):
        super().__init__()

        self.embeddings_layer = nn.Embedding(vocab_size, emb_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.lstm_layer = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim, bidirectional=True, num_layers=num_layers, batch_first=True)
        self.out_layer = nn.Linear(lstm_hidden_dim * 2, intents_count)

    def forward(self, inputs):
        projections = self.embeddings_layer(inputs)
        _, (final_hidden_state, _) = self.lstm_layer(projections)
        # print(output.shape)
        hidden = self.dropout(torch.cat([final_hidden_state[0], final_hidden_state[1]], dim=1))
        output = self.out_layer(hidden)
        return output

In [5]:
class ModelTrainer():
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        
    def on_epoch_begin(self, is_train, name, batches_count):
        """
        Initializes metrics
        """
        self.epoch_loss = 0
        self.correct_count, self.total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        
        self.model.train(is_train)
        
    def on_epoch_end(self):
        """
        Outputs final metrics
        """
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.correct_count / self.total_count
        )
        
    def on_batch(self, batch):
        """
        Performs forward and (if is_train) backward pass with optimization, updates metrics
        """
        logits = self.model(batch.tokens.transpose(0, 1))

        loss = self.criterion(logits, batch.intent)
        
        predicted_intent = logits.argmax(dim=1)
        self.total_count += predicted_intent.size(0)
        self.correct_count += torch.sum(predicted_intent == batch.intent).item()

        if self.is_train:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
        self.epoch_loss += loss.item()
        
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(trainer, data_iter, is_train, name=None):
    trainer.on_epoch_begin(is_train, name, batches_count=len(data_iter))
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=trainer.batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):
                batch_progress = trainer.on_batch(batch)

                progress_bar.update()
                progress_bar.set_description(batch_progress)
                
            epoch_progress = trainer.on_epoch_end()
            progress_bar.set_description(epoch_progress)
            progress_bar.refresh()

            
def fit(trainer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        do_epoch(trainer, train_iter, is_train=True, name=name_prefix + 'Train:')
        
        if not val_iter is None:
            do_epoch(trainer, val_iter, is_train=False, name=name_prefix + '  Val:')

In [6]:
model = IntentClassifierModel(vocab_size=len(tokens_field.vocab), intents_count=len(intent_field.vocab)).to(DEVICE)

criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters())

trainer = ModelTrainer(model, criterion, optimizer)

fit(trainer, train_iter, epochs_count=30, val_iter=val_iter)

torch.save(trainer, 'IntentClassifierModel.pt')

[1 / 30] Train: Loss = 0.85578, Accuracy = 80.42%: 100%|██████████| 140/140 [00:01<00:00, 78.96it/s]
[1 / 30]   Val: Loss = 0.50997, Accuracy = 86.20%: 100%|██████████| 4/4 [00:00<00:00, 65.57it/s]
[2 / 30] Train: Loss = 0.33923, Accuracy = 91.54%: 100%|██████████| 140/140 [00:01<00:00, 88.33it/s]
[2 / 30]   Val: Loss = 0.35395, Accuracy = 90.80%: 100%|██████████| 4/4 [00:00<00:00, 85.10it/s]
[3 / 30] Train: Loss = 0.22028, Accuracy = 93.84%: 100%|██████████| 140/140 [00:03<00:00, 40.09it/s]
[3 / 30]   Val: Loss = 0.27404, Accuracy = 93.60%: 100%|██████████| 4/4 [00:00<00:00, 28.99it/s]
[4 / 30] Train: Loss = 0.14592, Accuracy = 96.63%: 100%|██████████| 140/140 [00:03<00:00, 36.89it/s]
[4 / 30]   Val: Loss = 0.16378, Accuracy = 96.60%: 100%|██████████| 4/4 [00:00<00:00, 28.99it/s]
[5 / 30] Train: Loss = 0.09971, Accuracy = 97.54%: 100%|██████████| 140/140 [00:03<00:00, 36.03it/s]
[5 / 30]   Val: Loss = 0.16109, Accuracy = 96.40%: 100%|██████████| 4/4 [00:00<00:00, 29.41it/s]
[6 / 30] T

In [7]:
do_epoch(trainer, test_iter, is_train=False, name='Test:')

Test: Loss = 0.30393, Accuracy = 94.74%: 100%|██████████| 7/7 [00:00<00:00, 81.38it/s]


In [ ]:
DEVICE